In [2]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from dotenv import load_dotenv
import os


In [3]:
load_dotenv(dotenv_path = "password.env") 

# MONGO_URI example: MONGO_URI=mongodb://localhost:27017/

mongo_uri = os.getenv("mongo_uri")
mongo_database = os.getenv("mongo_database")

client = MongoClient(mongo_uri)

db = client[mongo_database]

In [ ]:
# Loading in csv data
patrons_data = pd.read_csv('patrons.csv')
inventory = pd.read_csv('inventory.csv')
books = pd.read_csv('books.csv')
authors = pd.read_csv('authors.csv')
loans = pd.read_csv('loans.csv')
books['isbn'] = books['isbn'].astype(str)
inventory['isbn'] = inventory['isbn'].astype(str)
authors['author_id'] = authors['author_id'].astype('int64')
books['author_id'] = books['author_id'].astype('int64')

# Converting to dictionaries
patrons_dict = patrons_data.to_dict('records')
loans_dict = loans.to_dict('records')
books_dict = books.to_dict('records')
inventory_dict = inventory.to_dict('records')
authors_dict = authors.to_dict('records')

# Adding authors as a subset of books
for item in books_dict:
    item['author'] = {}
for item in authors_dict:
    author_id = item['author_id']
    for book in books_dict:
        if book['author_id'] == author_id:
            book['author'] = item
            item.pop('author_id')
            break

# Merging books' meta data with inventoried books
for item in books_dict:
    isbn = item['isbn']
    for book in inventory_dict:
        if book['isbn'] == isbn:
            book['title'] = item['title']
            book['pub_year'] = item['pub_year']
            book['genre'] = item['genre']
            book['overdue_fee_rate'] = item['overdue_fee_rate']
            book['author'] = item['author']

# Adding inventories books as a subset of loans
for item in loans_dict:
    item['book'] = {}
for item in inventory_dict:
    book_id = item['book_id']
    for loan in loans_dict:
        if loan['book_ID'] == book_id:
            loan['book'] = item
    item.pop('book_id')

# Adding loans as a subset of patrons
for item in patrons_dict:
    item['loan_history'] = []
for item in loans_dict:
    patron_id = item['patron_ID']
    for patron in patrons_dict:
        if patron['patron_id'] == patron_id:
            patron['loan_history'].append(item)
            item.pop("patron_ID")
            item.pop("loan_ID")
            item.pop("book_ID")
            break

# Getting rid of `patron_id` variable in patrons
for item in patrons_dict:
    item.pop('patron_id', None)

# Adding patron objects to MongoDB
patrons = db.patrons
patrons.insert_many(patrons_dict)
patrons_dict
print(db.name)

# Delete current objects to repopulate if necessary
# patrons.delete_many({})

Forge_Week_2


[{'isbn': '9780140186390',
  'title': 'East of Eden',
  'pub_year': 1952,
  'genre': 'Classic',
  'overdue_fee_rate': 0.25,
  'author_id': 32,
  'author': {'first_name': 'John',
   'last_name': 'Steinbeck',
   'nationality': 'American',
   'language': 'English',
   'birth_year': '1902',
   'biography': 'John Steinbeck was a Nobel Prize-winning American author known for "The Grapes of Wrath", "Of Mice and Men", and "East of Eden". '}},
 {'isbn': '9780679720201',
  'title': 'The Stranger',
  'pub_year': 1942,
  'genre': 'Philosophy',
  'overdue_fee_rate': 0.25,
  'author_id': 3,
  'author': {'first_name': 'George',
   'last_name': 'Orwell',
   'nationality': 'British',
   'language': 'English',
   'birth_year': '1903',
   'biography': "George Orwell is best known for his novels '1984' and 'Animal Farm'."}},
 {'isbn': '9781250251305',
  'title': 'Miracle Creek',
  'pub_year': 2016,
  'genre': 'Thriller',
  'overdue_fee_rate': 0.5,
  'author_id': 2,
  'author': {'first_name': 'Albert',
   